#### Atividade1 : Implementando um Perceptron

Discente: Luiz Henrique Pereira Niero
Docente: Prof. Dr. LucasC. Ribas
PPGCC - Unesp - 2025
Disciplina: Rdes Neurais Artificiais

In [97]:
import pandas as pd
import numpy as np
import random

In [98]:
dataset_treinamento = 'datasets\\train_dataset1.csv'
dataset_testes = 'datasets\\test_dataset1.csv'

df_train_loaded = pd.read_csv(dataset_treinamento)
df_test_loaded = pd.read_csv(dataset_testes)

x_train = df_train_loaded.drop('label', axis=1).values
y_train = df_train_loaded['label'].values

x_test = df_test_loaded.drop('label', axis=1).values
y_test = df_test_loaded['label'].values

In [99]:
x_train

array([[ 2.54018315e-01, -4.02390597e-01],
       [ 1.80882656e-01,  5.10887202e-01],
       [ 2.27152011e-01,  4.67714858e-01],
       [-1.22832023e-01,  5.02634860e-01],
       [ 4.37192550e-01, -5.79591771e-01],
       [ 4.44119576e-01,  3.47945326e-01],
       [-4.64392658e-01,  4.49186211e-01],
       [-4.59897128e-01,  4.45262550e-01],
       [-4.86362688e-01,  3.79272165e-01],
       [ 9.54947621e-01,  2.23149772e-01],
       [-2.02775800e-01,  3.29165464e-01],
       [-1.01860222e+00,  6.41328317e-01],
       [ 4.71100045e-01,  6.17753181e-01],
       [-1.04807786e+00, -1.82686806e+00],
       [ 3.31793822e-01,  2.52452115e-01],
       [ 1.00537805e-01, -8.04403853e-01],
       [-5.55574552e-01,  5.42136981e-01],
       [-5.36420754e-01, -1.33720555e+00],
       [ 1.12001905e-01,  5.24914452e-01],
       [-6.50005573e-01,  7.15095693e-01],
       [ 1.02424668e+00,  1.67321235e-01],
       [ 1.62595838e+00,  1.06203590e+00],
       [ 4.01642181e-01,  5.70783227e-01],
       [ 4.

In [100]:
class Perceptron:
    def __init__ (self, x_train, y_train):
        self.x_train = x_train
        self.y_train = y_train
        self.quantidade_elementos_treino = x_train.shape[0]
        self.quantidade_atributos = x_train.shape[1] 
        self.w = np.random.uniform(0, 0.2, size=self.quantidade_atributos)
        self.limiar = random.uniform(0, 0.2) # ponto flutuante entre 0 e 0.2
        self.tx_aprendizagem = 0.1
        self.qtde_epocas = 50
        self.erros = 0
        self.w_epocas = []
        self.bias_epocas = []
        self.erros_epocas = []

    # Ajusta os pesos para uma linha da matriz de entrada (aqui uma linha corresponde a um elemento x do conjunto X de treinamento)
    # Retorna 1 se a predição foi correta e 0 se a predição foi errada
    def treina_elemento(self, indice):
        gux = 0
        wx = 0
        y = self.y_train[indice]
    
        # Itera por cada elemento i de xi e wi e faz a soma em wx.
        for i in range(self.quantidade_atributos):
            wxi = self.x_train[indice][i] * self.w[i]
            wx += wxi
    
        wx = wx + self.limiar
        
        if wx >= 0:
            gux = 1
        else:
            gux = -1
    
        if y == gux:
            return 1
        else:
            diferenca = y - gux
            ajuste_base = self.tx_aprendizagem * diferenca

            # Ajusta cada elemento do vetor W
            for j in range (self.quantidade_atributos):
                self.w[j] = self.w[j] + ajuste_base * self.x_train[indice][j]
            
            # Ajusta o bias
            self.limiar = self.limiar + ajuste_base
            return 0


    def treina_epoca(self, epoca):
        acertos_da_epoca = 0
        erros_da_epoca = 0
        
        for i in range(self.quantidade_elementos_treino):
            acertos_da_epoca += self.treina_elemento(i)

        # salvando os pesos da época.
        self.w_epocas.append(self.w.copy())
        self.bias_epocas.append(self.limiar.copy())
        erros_da_epoca = self.quantidade_elementos_treino - acertos_da_epoca
        self.erros_epocas.append(erros_da_epoca)
        
        self.erros = erros_da_epoca
        print(f"Época {epoca}. Acertos: {acertos_da_epoca} Erros: {erros_da_epoca}. Taxa de acertos: {acertos_da_epoca / self.quantidade_elementos_treino}")          


    def treina(self):
        for epoca in range(self.qtde_epocas):
            self.treina_epoca(epoca)

In [101]:
# Adiciono o BIAS como um dos elementos do conjunto de treinamento

In [102]:
# Recebe um vetor X e o valor previsto Y e atualiza os parâmetros W e Bias.
perceptron = Perceptron(x_train, y_train)
perceptron.treina()

print("\r\n\r\n --- Resultados do treinamento --- ")
melhor_epoca =  perceptron.erros_epocas.index(min(perceptron.erros_epocas))
melhor_w = perceptron.w_epocas[melhor_epoca]
melhor_bias = perceptron.bias_epocas[melhor_epoca]
erros_melhor_epoca = perceptron.erros_epocas[melhor_epoca]
tx_acerto_melhor_epoca = (perceptron.quantidade_elementos_treino - erros_melhor_epoca) / perceptron.quantidade_elementos_treino

print(f"Melhor época: {melhor_epoca}")
print(f"Quantidade de erros: {erros_melhor_epoca}")

print(f"Taxa de acertos da melhor época: {tx_acerto_melhor_epoca}")
print(f"Melhores pesos: {melhor_w}")
print(f"Melhor bias: {melhor_bias}")

Época 0. Acertos: 122 Erros: 18. Taxa de acertos: 0.8714285714285714
Época 1. Acertos: 131 Erros: 9. Taxa de acertos: 0.9357142857142857
Época 2. Acertos: 130 Erros: 10. Taxa de acertos: 0.9285714285714286
Época 3. Acertos: 135 Erros: 5. Taxa de acertos: 0.9642857142857143
Época 4. Acertos: 132 Erros: 8. Taxa de acertos: 0.9428571428571428
Época 5. Acertos: 136 Erros: 4. Taxa de acertos: 0.9714285714285714
Época 6. Acertos: 136 Erros: 4. Taxa de acertos: 0.9714285714285714
Época 7. Acertos: 136 Erros: 4. Taxa de acertos: 0.9714285714285714
Época 8. Acertos: 131 Erros: 9. Taxa de acertos: 0.9357142857142857
Época 9. Acertos: 136 Erros: 4. Taxa de acertos: 0.9714285714285714
Época 10. Acertos: 137 Erros: 3. Taxa de acertos: 0.9785714285714285
Época 11. Acertos: 136 Erros: 4. Taxa de acertos: 0.9714285714285714
Época 12. Acertos: 136 Erros: 4. Taxa de acertos: 0.9714285714285714
Época 13. Acertos: 131 Erros: 9. Taxa de acertos: 0.9357142857142857
Época 14. Acertos: 136 Erros: 4. Taxa de a

In [103]:
melhor_w

array([-0.96322159,  0.88320722])